<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas_postgres/pandas_postgres_hard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

185. Department Top Three Salaries

https://leetcode.com/problems/department-top-three-salaries/description/

```sql

SELECT d.name as Department, sub.name as Employee, sub.salary as Salary
FROM (
SELECT
    e.departmentId , e.name, e.salary,
    dense_rank() OVER (PARTITION BY departmentId ORDER BY salary DESC) as rnk
FROM Employee e) sub
LEFT JOIN Department d ON d.id = sub.departmentId
WHERE sub.rnk < 4
```

In [ ]:
data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1], [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    return employee.assign(rnk = employee.groupby('departmentId').salary.rank(method='dense', ascending=False))\
                    .query('rnk <= 3', engine='python')\
                    .merge(department, how='left', left_on='departmentId', right_on='id', suffixes=['_emp', '_dpt'])\
                    [['name_dpt', 'name_emp', 'salary']].rename({'name_dpt': 'Department', 'name_emp': 'Employee', 'salary': 'Salary'}, axis=1)

262. Trips and Users

https://leetcode.com/problems/trips-and-users/description/

```sql
WITH banned AS (
    SELECT users_id FROM Users WHERE banned = 'Yes'
)
SELECT
request_at as Day,
ROUND(1 - count(id) FILTER (WHERE t.status NOT IN ('cancelled_by_driver', 'cancelled_by_client'))*1.0 / count(id), 2) as "Cancellation Rate"

FROM Trips t
WHERE t.request_at  BETWEEN '2013-10-01' and '2013-10-03' AND t.client_id NOT IN (SELECT users_id FROM banned) AND t.driver_id NOT IN (SELECT users_id FROM banned)
GROUP BY request_at
HAVING count(t.id) > 0
```

The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Write a solution to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03" with at least one trip. Round Cancellation Rate to two decimal points.

Return the result table in any order.



In [ ]:
data = [['1', '1', '10', '1', 'completed', '2013-10-01'], ['2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01'], ['3', '3', '12', '6', 'completed', '2013-10-01'], ['4', '4', '13', '6', 'cancelled_by_client', '2013-10-01'], ['5', '1', '10', '1', 'completed', '2013-10-02'], ['6', '2', '11', '6', 'completed', '2013-10-02'], ['7', '3', '12', '6', 'completed', '2013-10-02'], ['8', '2', '12', '12', 'completed', '2013-10-03'], ['9', '3', '10', '12', 'completed', '2013-10-03'], ['10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03']]
trips = pd.DataFrame(data, columns=['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at']).astype({'id':'Int64', 'client_id':'Int64', 'driver_id':'Int64', 'city_id':'Int64', 'status':'object', 'request_at':'object'})

data = [['1', 'No', 'client'], ['2', 'Yes', 'client'], ['3', 'No', 'client'], ['4', 'No', 'client'], ['10', 'No', 'driver'], ['11', 'No', 'driver'], ['12', 'No', 'driver'], ['13', 'No', 'driver']]
users = pd.DataFrame(data, columns=['users_id', 'banned', 'role']).astype({'users_id':'Int64', 'banned':'object', 'role':'object'})

In [ ]:
def trips_and_users(trips: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    banned_ids = users[users.banned == 'Yes'].users_id.values
    df = trips[(trips.request_at.between('2013-10-01', '2013-10-03')) &
        (~trips.client_id.isin(banned_ids)) &
        (~trips.driver_id.isin(banned_ids))]
    df = df.assign(cancelled = np.where(df.status.str.match('cancelled'), 1, 0))\
    .groupby('request_at')['cancelled'].agg(['sum', 'size']).reset_index().rename({"request_at": "Day"}, axis=1)


    df['Cancellation Rate'] = (df['sum']/df['size']).round(2)
    return df[['Day','Cancellation Rate']]

601. Human Traffic of Stadium

https://leetcode.com/problems/human-traffic-of-stadium/description/

```sql
-- Write your PostgreSQL query statement below
SELECT
id, visit_date, people
FROM (
SELECT
*,
LEAD(id) OVER (ORDER BY id) as ld,
LAG(id) OVER (ORDER BY id) as lg,
LEAD(id, 2) OVER (ORDER BY id) as ld2,
LAG(id, 2) OVER (ORDER BY id) as lg2
FROM Stadium
WHERE people   >= 100) sub
WHERE (id - lg = 1 AND ld - id = 1) OR
      (id - lg = 1 AND id - lg2 = 2) OR
      (ld - id = 1 AND ld2 - id = 2)
ORDER BY visit_date
```

Write a solution to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The result format is in the following example.

In [ ]:
data = [[1, '2017-01-01', 10], [2, '2017-01-02', 109], [3, '2017-01-03', 150], [4, '2017-01-04', 99], [5, '2017-01-05', 145], [6, '2017-01-06', 1455], [7, '2017-01-07', 199], [8, '2017-01-09', 188]]
stadium = pd.DataFrame(data, columns=['id', 'visit_date', 'people']).astype({'id':'Int64', 'visit_date':'datetime64[ns]', 'people':'Int64'})

In [ ]:
def human_traffic(stadium: pd.DataFrame) -> pd.DataFrame:
    filtered = stadium[stadium.people.ge(100)]
    return filtered.assign(lg1_diff = filtered.id.diff(1), ld1_diff = - filtered.id.diff(-1), lg2_diff = filtered.id.diff(2), ld2_diff = - filtered.id.diff(-2))\
                  .query("(lg1_diff == 1 & ld1_diff == 1) | \
                          (ld1_diff == 1 & ld2_diff == 2) | \
                          (lg1_diff == 1 & lg2_diff == 2)", engine='python')\
                          .iloc[:,0:3].sort_values(by='visit_date')

3374. First Letter Capitalization II

https://leetcode.com/problems/first-letter-capitalization-ii/description/

```sql
SELECT
	content_id,
	content_text AS original_text,
  initcap(content_text)   AS converted_text
FROM user_content
```

In [ ]:
data = [[1, 'hello world of SQL'], [2, 'the QUICK-brown fox'], [3, 'modern-day DATA science'], [4, 'web-based FRONT-end development']]
user_content = pd.DataFrame(data, columns=['content_id', 'content_text'])

In [ ]:
def capitalize_content(user_content: pd.DataFrame) -> pd.DataFrame:
    user_content['converted_text'] = user_content.content_text.apply(lambda x: x.title())
    return user_content.rename({'content_text': 'original_text'}, axis=1)

569. Median Employee Salary

https://leetcode.com/problems/median-employee-salary/description/

```sql
SELECT id, company, salary FROM (
SELECT
company,
id, salary,
row_number() OVER (PARTITION BY company ORDER BY salary DESC, id DESC) -
row_number() OVER (PARTITION BY company ORDER BY salary ASC, id ASC) as rn_delta,
count(*) OVER (PARTITION BY company) AS  cnt
FROM Employee
 ) sub
--WHERE rn_delta <= 1 - (cnt * 1.0) % 2 AND rn_delta >= -1 + (cnt * 1.0) % 2
WHERE rn_delta <= 1  AND rn_delta >= -1
```

571. Find Median Given Frequency of Numbers

https://leetcode.com/problems/find-median-given-frequency-of-numbers/description/

```sql
SELECT
percentile_cont(0.5) WITHIN GROUP (ORDER BY num) as median FROM (
SELECT num, generate_series(1, frequency)  FROM Numbers) sub
```

579. Find Cumulative Salary of an Employee

https://leetcode.com/problems/find-median-given-frequency-of-numbers/

```sql
WITH t as (
SELECT
id, generate_series(1, mm) as month
FROM (
SELECT
id, max(month) - 1 as mm
FROM Employee
GROUP BY id) sub
ORDER BY 1, 2),

analytical_t as (
SELECT
id, month, COALESCE(salary, 0) as salary
FROM t
LEFT JOIN Employee USING (id, month)),

full_table as (
SELECT
id, month,
sum(salary) OVER (PARTITION BY id ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as Salary
FROM analytical_t)

SELECT f.id, f.month, f.Salary FROM full_table f
INNER JOIN Employee e USING(id, month)
ORDER BY 1, 2 DESC
```

```sql
SELECT e.id, e.month,
    SUM(e2.Salary) as Salary
FROM Employee e
JOIN Employee e2 ON (e.Id = e2.Id AND e.Month >= e2.Month AND (e.Month - e2.Month <= 2))
WHERE e.Month < (SELECT MAX(Month) from Employee where Id = e.Id)
GROUP BY e.Id, e.Month
ORDER BY e.Id, e.Month DESC
```